In [1]:
# Configure the necessary Spark environment
import os
import sys

spark_home = os.environ.get('SPARK_HOME', None)
sys.path.insert(0, spark_home + "/python")

# Add the py4j to the path.
# You may need to change the version number to match your install -- currently using spark 2.4
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))

#os.environ['PYSPARK_SUBMIT_ARGS']="--jars /work/ericr/spark/sparkdev/postgresql.jar --executor-memory 40g --executor-cores 16 pyspark-shell"

# Initialize PySpark to predefine the SparkContext variable 'sc'
#execfile(os.path.join(spark_home, 'python/pyspark/shell.py'))

In [15]:
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import flatten, explode, col
from pyspark.sql.functions import row_number, monotonically_increasing_id
from pyspark.sql import Window
from pyspark.sql.types import StructField
from pyspark.sql.types import StructType
import json



In [3]:
spark = SparkSession.builder.appName('Basic').config("spark.executor.extraJavaOptions","--executor-memory 40G --executor-cores 40 --driver-class-path $SPARK_HOME/postgresql.jar").getOrCreate()

In [4]:
sqlContext = SQLContext(spark.sparkContext)

In [5]:
spark.conf.set("spark.sql.caseSensitive","true")

In [6]:
dbconfig = {"url": "jdbc:postgresql://localhost/qxedb",
            "dbtable": "gateway_eventnotification",
            "user": "hermes",
            "password": "mysecret",
            "driver": "org.postgresql.Driver"}

In [7]:
df = spark.read.jdbc(url="jdbc:postgresql://localhost/qxedb", 
                      table="gateway_eventnotification",
                      properties=dbconfig)

In [8]:
sqlcontext = SQLContext(spark.sparkContext)

In [9]:
payload_df = sqlcontext.read.json(df.rdd.map(lambda r: r.payload))

In [10]:
sch = payload_df.schema.jsonValue()

In [11]:
sch_str = json.dumps(sch)

In [12]:
sch2_str = sch_str.replace("onDurationTime (sec)","onDurationTime_sec")

In [13]:
sch2 = json.loads(sch2_str)

In [16]:
s2=StructType.fromJson(sch2)

In [17]:
payload_df = sqlcontext.createDataFrame(payload_df.rdd, s2)

In [18]:
payload_df.printSchema()

root
 |-- data_type: string (nullable = true)
 |-- events: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- dateTime: string (nullable = true)
 |    |    |-- eventCategory: string (nullable = true)
 |    |    |-- eventDetail: struct (nullable = true)
 |    |    |    |-- brand: string (nullable = true)
 |    |    |    |-- connectionInfo: struct (nullable = true)
 |    |    |    |    |-- localIpv4Address: struct (nullable = true)
 |    |    |    |    |    |-- addr: string (nullable = true)
 |    |    |    |    |    |-- subnetMask: string (nullable = true)
 |    |    |    |    |-- sourceInSameSubnet: string (nullable = true)
 |    |    |    |    |-- sourceIpv4Address: struct (nullable = true)
 |    |    |    |    |    |-- addr: string (nullable = true)
 |    |    |    |    |    |-- subnetMask: string (nullable = true)
 |    |    |    |    |-- sourceIpv6Address: struct (nullable = true)
 |    |    |    |    |    |-- addr: string (nullable = true)
 |   

In [19]:
payload_df = payload_df.withColumn("index",row_number().over(Window.orderBy(monotonically_increasing_id()))-1)

In [20]:
x = payload_df.select(["index",explode(payload_df.events).alias("event"),
                       "henq_metadata",
                       "links",
                       "metadata",
                       "originator",
                       "payloadManifestJec",
                       "subscriptionId",
                       "system_check_id",
                       "version"])

In [ ]:
#x2 = x.select(["index",
#                       "event",
#                       "henq_metadata",
#                       explode(payload_df.links).alias("link"),
#                       "metadata",
#                       "originator",
#                       "payloadManifestJec",
#                       "subscriptionId",
#                       "system_check_id",
#                       "version"])

In [21]:
x=x.na.drop(subset=["event.sequenceNumber"])
x=x.na.drop(subset=["event.eventDetail.serialNumber"])

In [22]:
x.show(truncate=18)

+-----+------------------+------------------+------------------+------------------+------------------+------------------+------------------+---------------+-------+
|index|             event|     henq_metadata|             links|          metadata|        originator|payloadManifestJec|    subscriptionId|system_check_id|version|
+-----+------------------+------------------+------------------+------------------+------------------+------------------+------------------+---------------+-------+
|  410|[2018-03-15T10:...|[2018-03-15 20:...|[[/cdm/telemetr...|              null|              null|              null|f2dece9d-3087-4...|           null|  1.0.0|
|  410|[2018-03-15T10:...|[2018-03-15 20:...|[[/cdm/telemetr...|              null|              null|              null|f2dece9d-3087-4...|           null|  1.0.0|
|  410|[2018-03-15T10:...|[2018-03-15 20:...|[[/cdm/telemetr...|              null|              null|              null|f2dece9d-3087-4...|           null|  1.0.0|
|  668|[20

In [ ]:
#x=x.withColumnRenamed("onDurationTime (sec)","onDurationTime")
x.write.parquet("/work/ericr/payload.parquet",compression="gzip",mode="overwrite")


In [ ]:
# now read it back in

In [23]:
dfp = spark.read.parquet("/work/ericr/payload.parquet")

In [24]:
dfp.printSchema()

root
 |-- index: integer (nullable = true)
 |-- event: struct (nullable = true)
 |    |-- dateTime: string (nullable = true)
 |    |-- eventCategory: string (nullable = true)
 |    |-- eventDetail: struct (nullable = true)
 |    |    |-- brand: string (nullable = true)
 |    |    |-- connectionInfo: struct (nullable = true)
 |    |    |    |-- localIpv4Address: struct (nullable = true)
 |    |    |    |    |-- addr: string (nullable = true)
 |    |    |    |    |-- subnetMask: string (nullable = true)
 |    |    |    |-- sourceInSameSubnet: string (nullable = true)
 |    |    |    |-- sourceIpv4Address: struct (nullable = true)
 |    |    |    |    |-- addr: string (nullable = true)
 |    |    |    |    |-- subnetMask: string (nullable = true)
 |    |    |    |-- sourceIpv6Address: struct (nullable = true)
 |    |    |    |    |-- addr: string (nullable = true)
 |    |    |    |-- sourceMacAddress: string (nullable = true)
 |    |    |-- connectionState: string (nullable = true)
 |    

In [29]:
dfp.orderBy("event.eventDetail.sequenceNumber",ascending=True).select("event.eventDetail.sequenceNumber").show(20)

+--------------+
|sequenceNumber|
+--------------+
|          null|
|          null|
|          null|
|          null|
|          null|
|          null|
|          null|
|          null|
|          null|
|          null|
|          null|
|          null|
|          null|
|          null|
|          null|
|          null|
|          null|
|          null|
|          null|
|          null|
+--------------+
only showing top 20 rows



In [30]:
dfp.show()

+-----+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+--------------------+---------------+-------+
|index|               event|       henq_metadata|               links|            metadata|          originator|payloadManifestJec|      subscriptionId|system_check_id|version|
+-----+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+--------------------+---------------+-------+
|  410|[2018-03-15T10:52...|[2018-03-15 20:55...|[[/cdm/telemetry/...|                null|                null|              null|f2dece9d-3087-485...|           null|  1.0.0|
|  410|[2018-03-15T10:52...|[2018-03-15 20:55...|[[/cdm/telemetry/...|                null|                null|              null|f2dece9d-3087-485...|           null|  1.0.0|
|  410|[2018-03-15T10:52...|[2018-03-15 20:55...|[[/cdm/telemetry/...|                null|                null|   

In [31]:
dfp.count()

22261

In [ ]:
d=dfp.select("eventCategory")

In [ ]:
d.show()

In [ ]:
from pyspark.sql.functions import row_number, monotonically_increasing_id
from pyspark.sql import Window

In [ ]:
dr = dfp.withColumn("index",row_number().over(Window.orderBy(monotonically_increasing_id()))-1)

In [ ]:
dr.show(truncate=18)